<a href="https://colab.research.google.com/github/isshii/de4test/blob/master/deepXplore_for_kaggle_model_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
###########################

In [2]:
import argparse

%tensorflow_version 1.x

# from keras.applications.mobilenet import MobileNet
# from keras.applications.densenet import DenseNet201
# from keras.applications.nasnet import NASNetMobile


from keras.layers import Input
#from scipy.misc import imsave  
import imageio

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

#from utils import *


Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13809135235432608505, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14830336018363494975
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 81958342282542278
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15956099072
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2440277145442589903
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [0]:
# read the parameter
# argument parsing
parser = argparse.ArgumentParser(description='Main function for difference-inducing input generation in MNIST dataset')
parser.add_argument('transformation', help="realistic transformation type", choices=['light', 'occl', 'blackout'])
parser.add_argument('weight_diff', help="weight hyperparm to control differential behavior", type=float)
parser.add_argument('weight_nc', help="weight hyperparm to control neuron coverage", type=float)
parser.add_argument('step', help="step size of gradient descent", type=float)
parser.add_argument('seeds', help="number of seeds of input", type=int)
parser.add_argument('grad_iterations', help="number of iterations of gradient descent", type=int)
parser.add_argument('threshold', help="threshold for determining neuron activated", type=float)
parser.add_argument('-t', '--target_model', help="target model that we want it predicts differently",
                    choices=[0, 1, 2], default=0, type=int)
parser.add_argument('-sp', '--start_point', help="occlusion upper left corner coordinate", default=(0, 0), type=tuple)
parser.add_argument('-occl_size', '--occlusion_size', help="occlusion size", default=(10, 10), type=tuple)

#args = parser.parse_args([])
args = parser.parse_args(["occl", "0.1", "0.1", "1", "100", "100", "0.1"])

In [0]:
import random
from collections import defaultdict

import numpy as np
from keras import backend as K
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import Model
from keras.preprocessing import image
import os, re


def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    input_img_data = image.img_to_array(img)
    input_img_data = np.expand_dims(input_img_data, axis=0)
    input_img_data = preprocess_input(input_img_data)  # final input shape = (1,224,224,3)
    return input_img_data


def deprocess_image(x):
    x = x.reshape((224, 224, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def decode_label(pred):
    return decode_predictions(pred)[0][0][1]


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)


def constraint_occl(gradients, start_point, rect_shape):
    new_grads = np.zeros_like(gradients)
    new_grads[:, start_point[0]:start_point[0] + rect_shape[0],
    start_point[1]:start_point[1] + rect_shape[1]] = gradients[:, start_point[0]:start_point[0] + rect_shape[0],
                                                     start_point[1]:start_point[1] + rect_shape[1]]
    return new_grads


def constraint_light(gradients):
    new_grads = np.ones_like(gradients)
    grad_mean = 1e4 * np.mean(gradients)
    return grad_mean * new_grads


def constraint_black(gradients, rect_shape=(10, 10)):
    start_point = (
        random.randint(0, gradients.shape[1] - rect_shape[0]), random.randint(0, gradients.shape[2] - rect_shape[1]))
    new_grads = np.zeros_like(gradients)
    patch = gradients[:, start_point[0]:start_point[0] + rect_shape[0], start_point[1]:start_point[1] + rect_shape[1]]
    if np.mean(patch) < 0:
        new_grads[:, start_point[0]:start_point[0] + rect_shape[0],
        start_point[1]:start_point[1] + rect_shape[1]] = -np.ones_like(patch)
    return new_grads


def init_coverage_tables(model1, model2, model3):
    model_layer_dict1 = defaultdict(bool)
    model_layer_dict2 = defaultdict(bool)
    model_layer_dict3 = defaultdict(bool)
    init_dict(model1, model_layer_dict1)
    init_dict(model2, model_layer_dict2)
    init_dict(model3, model_layer_dict3)
    return model_layer_dict1, model_layer_dict2, model_layer_dict3


def init_dict(model, model_layer_dict):
    for layer in model.layers:
        if 'flatten' in layer.name or 'input' in layer.name:
            continue
        for index in range(layer.output_shape[-1]):
            model_layer_dict[(layer.name, index)] = False


def neuron_to_cover(model_layer_dict):
    not_covered = [(layer_name, index) for (layer_name, index), v in list(model_layer_dict.items()) if not v]
    if not_covered:
        layer_name, index = random.choice(not_covered)
    else:
        layer_name, index = random.choice(list(model_layer_dict.keys()))
    return layer_name, index


def neuron_covered(model_layer_dict):
    covered_neurons = len([v for v in list(model_layer_dict.values()) if v])
    total_neurons = len(model_layer_dict)
    return covered_neurons, total_neurons, covered_neurons / float(total_neurons)


def scale(intermediate_layer_output, rmax=1, rmin=0):
    X_std = (intermediate_layer_output - intermediate_layer_output.min()) / (
        intermediate_layer_output.max() - intermediate_layer_output.min())
    X_scaled = X_std * (rmax - rmin) + rmin
    return X_scaled


def update_coverage(input_data, model, model_layer_dict, threshold=0):
    layer_names = [layer.name for layer in model.layers if
                   'flatten' not in layer.name and 'input' not in layer.name]

    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=[model.get_layer(layer_name).output for layer_name in layer_names])
    intermediate_layer_outputs = intermediate_layer_model.predict(input_data)

    for i, intermediate_layer_output in enumerate(intermediate_layer_outputs):
        scaled = scale(intermediate_layer_output[0])
        for num_neuron in range(scaled.shape[-1]):
            if np.mean(scaled[..., num_neuron]) > threshold and not model_layer_dict[(layer_names[i], num_neuron)]:
                model_layer_dict[(layer_names[i], num_neuron)] = True


def full_coverage(model_layer_dict):
    if False in list(model_layer_dict.values()):
        return False
    return True


def fired(model, layer_name, index, input_data, threshold=0):
    intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    intermediate_layer_output = intermediate_layer_model.predict(input_data)[0]
    scaled = scale(intermediate_layer_output)
    if np.mean(scaled[..., index]) > threshold:
        return True
    return False


def diverged(predictions1, predictions2, predictions3, target):
    #     if predictions2 == predictions3 == target and predictions1 != target:
    if not predictions1 == predictions2 == predictions3:
        return True
    return False

#add
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

In [0]:
%%time
from keras import models
from keras import optimizers
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19

# input image dimensions
img_rows, img_cols = 224, 224
input_shape = (img_rows, img_cols, 3)
input_tensor = Input(shape=input_shape)

vggmodel = VGG16(weights='imagenet', include_top=True)

for layers in (vggmodel.layers)[:19]:
    print(layers)
    layers.trainable = False
X= vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model1 = Model(input = vggmodel.input, output = predictions)
model1.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model1.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
vgg16_weights = '/content/gdrive/My Drive/deepxplore/models/good_vgg16.h5'

# model1 = VGG19(weights=vgg19_weights)
# model2 = VGG16(weights=vgg16_weights)
# model3 = ResNet50(weights=resnet50_weights)

#model1 = VGG19(weights=vgg19_weights,input_tensor=input_tensor)
model1.load_weights(vgg16_weights)

In [0]:
# start gen inputs
#img_paths = image.list_pictures('./seeds', ext='JPEG')
img_paths = list_pictures('./gdrive/My Drive/deepxplore/ImageNet/seeds')
gen_img = preprocess_image(random.choice(img_paths))

In [0]:
pred1 = model1.predict(gen_img)
label1 = np.argmax(pred1[0])

In [0]:
print(pred1)

[[0.30713782 0.69286215]]


In [0]:
######################

In [0]:
%%time
from keras import models
from keras import optimizers
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19

# input image dimensions
img_rows, img_cols = 224, 224
input_shape = (img_rows, img_cols, 3)
input_tensor = Input(shape=input_shape)

vggmodel1 = VGG16(weights='imagenet',input_tensor=input_tensor,include_top=True)

for layers in (vggmodel1.layers)[:19]:
    print(layers)
    layers.trainable = False
X= vggmodel1.layers[-2].output
predictions1 = Dense(2, activation="softmax")(X)
model1 = Model(input = vggmodel1.input, output = predictions1)
model1.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model1.summary()

#vgg16_weights = '/content/gdrive/My Drive/deepxplore/models/good_vgg16.h5'
#model1.load_weights(vgg16_weights)

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
%%time
vggmodel2 = VGG16(weights='imagenet',input_tensor=input_tensor, include_top=True)

for layers in (vggmodel2.layers)[:19]:
    print(layers)
    layers.trainable = False
X= vggmodel2.layers[-2].output
predictions2 = Dense(2, activation="softmax")(X)
model2 = Model(input = vggmodel2.input, output = predictions2)
model2.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model2.summary()

vgg16_weights = '/content/gdrive/My Drive/deepxplore/models/good_vgg16.h5'
model2.load_weights(vgg16_weights)

Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


CPU times: user 2.7 s, sys: 555 ms, total: 3.25 s
Wall time: 3.72 s


In [0]:
%%time
vggmodel3 = VGG16(weights='imagenet',input_tensor=input_tensor, include_top=True)

for layers in (vggmodel3.layers)[:19]:
    print(layers)
    layers.trainable = False
X= vggmodel3.layers[-2].output
predictions3 = Dense(2, activation="softmax")(X)
model3 = Model(input = vggmodel3.input, output = predictions3)
model3.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model3.summary()

vgg16_weights = '/content/gdrive/My Drive/deepxplore/models/good_vgg16.h5'
model3.load_weights(vgg16_weights)

Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


CPU times: user 2.79 s, sys: 500 ms, total: 3.29 s
Wall time: 3.79 s


In [0]:
K.set_learning_phase(0)
model_layer_dict1, model_layer_dict2, model_layer_dict3 = init_coverage_tables(model1, model2, model3)


In [0]:
# start gen inputs
#img_paths = image.list_pictures('./seeds', ext='JPEG')
img_paths = list_pictures('./gdrive/My Drive/deepxplore/ImageNet/seeds')
gen_img = preprocess_image(random.choice(img_paths))
# print(gen_img)

In [0]:
gen_img = preprocess_image(random.choice(img_paths))
orig_img = gen_img.copy()
# first check if input already induces differences
pred1, pred2, pred3 = model1.predict(gen_img), model2.predict(gen_img), model3.predict(gen_img)
label1, label2, label3 = np.argmax(pred1[0]), np.argmax(pred2[0]), np.argmax(pred3[0])

In [0]:
print(label1, label2, label3)

0 0 0


In [0]:
print('input already causes different outputs: {}, {}, {}'.format(label1, label2 ,label3) + bcolors.ENDC)  

input already causes different outputs: 1, 1, 1


In [0]:
######################## 摂動を加えなくても予測が3つのモデルで異なる場合の処理 Start ########################
print('input already causes different outputs: {}, {}, {}'.format(label1, label2 ,label3) + bcolors.ENDC) 

update_coverage(gen_img, model1, model_layer_dict1, args.threshold)
update_coverage(gen_img, model2, model_layer_dict2, args.threshold)
update_coverage(gen_img, model3, model_layer_dict3, args.threshold)

print(bcolors.OKGREEN + 'covered neurons percentage %d neurons %.3f, %d neurons %.3f, %d neurons %.3f'
#        print('covered neurons percentage %d neurons %.3f, %d neurons %.3f, %d neurons %.3f'
      % (len(model_layer_dict1), neuron_covered(model_layer_dict1)[2], len(model_layer_dict2),
          neuron_covered(model_layer_dict2)[2], len(model_layer_dict3),
          neuron_covered(model_layer_dict3)[2]) + bcolors.ENDC)
#                 neuron_covered(model_layer_dict3)[2]))

averaged_nc = (neuron_covered(model_layer_dict1)[0] + neuron_covered(model_layer_dict2)[0] +
                neuron_covered(model_layer_dict3)[0]) / float(
    neuron_covered(model_layer_dict1)[1] + neuron_covered(model_layer_dict2)[1] +
    neuron_covered(model_layer_dict3)[
        1])
print(bcolors.OKGREEN + 'averaged covered neurons %.3f' % averaged_nc + bcolors.ENDC)
print('averaged covered neurons %.3f' % averaged_nc)

gen_img_deprocessed = deprocess_image(gen_img)

# save the result to disk
#imsave('./generated_inputs/' + 'already_differ_' + decode_label(pred1) + '_' + decode_label(
imageio.imwrite('./gdrive/My Drive/deepxplore/ImageNet/generated_inputs/' + 'already_differ_' + str(label1) + '_' + str(label2) + '_' + str(label3) + '.png',
        gen_img_deprocessed)
continue
######################## 摂動を加えなくても予測が3つのモデルで異なる場合の処理END ########################

input already causes different outputs: 1, 1, 1
covered neurons percentage 13890 neurons 0.143, 13890 neurons 0.143, 13890 neurons 0.143
averaged covered neurons 0.143
averaged covered neurons 0.143


SyntaxError: ignored

In [0]:
orig_label = label1
layer_name1, index1 = neuron_to_cover(model_layer_dict1)
layer_name2, index2 = neuron_to_cover(model_layer_dict2)
layer_name3, index3 = neuron_to_cover(model_layer_dict3)

In [0]:
print(orig_label,layer_name1, index1)
print(orig_label,layer_name2, index2)
print(orig_label,layer_name3, index3)

1 fc2 2957
1 block5_conv3 278
1 block3_conv3 236


In [0]:
layer_output = (loss1 + loss2 + loss3) + args.weight_nc * (loss1_neuron + loss2_neuron + loss3_neuron) 

In [0]:
layer_output

<tf.Tensor 'add_45:0' shape=() dtype=float32>

In [0]:
# for adversarial image generation
final_loss = K.mean(layer_output)

# we compute the gradient of the input picture wrt this loss
grads = normalize(K.gradients(final_loss, input_tensor)[0])#####  エラー箇所
print(grads)

Tensor("truediv_7:0", shape=(?, 224, 224, 3), dtype=float32)


In [0]:
######################## 予測が3つのモデルで同じ場合の処理 Start ########################
# if all label agrees
orig_label = label1
layer_name1, index1 = neuron_to_cover(model_layer_dict1)
layer_name2, index2 = neuron_to_cover(model_layer_dict2)
layer_name3, index3 = neuron_to_cover(model_layer_dict3)
# construct joint loss function
if args.target_model == 0:
    loss1 = -args.weight_diff * K.mean(model1.get_layer('dense_7').output[..., orig_label])
    loss2 = K.mean(model2.get_layer('dense_8').output[..., orig_label])
    loss3 = K.mean(model3.get_layer('dense_9').output[..., orig_label])
elif args.target_model == 1:
    loss1 = K.mean(model1.get_layer('dense_7').output[..., orig_label])
    loss2 = -args.weight_diff * K.mean(model2.get_layer('dense_8').output[..., orig_label])
    loss3 = K.mean(model3.get_layer('dense_9').output[..., orig_label])
elif args.target_model == 2:
    loss1 = K.mean(model1.get_layer('dense_7').output[..., label1])
    loss2 = K.mean(model2.get_layer('dense_8').output[..., orig_label])
    loss3 = -args.weight_diff * K.mean(model3.get_layer('dense_9').output[..., orig_label])
loss1_neuron = K.mean(model1.get_layer(layer_name1).output[..., index1])
loss2_neuron = K.mean(model2.get_layer(layer_name2).output[..., index2])
loss3_neuron = K.mean(model3.get_layer(layer_name3).output[..., index3])
layer_output = (loss1 + loss2 + loss3) + args.weight_nc * (loss1_neuron + loss2_neuron + loss3_neuron)    

# for adversarial image generation
final_loss = K.mean(layer_output)

# we compute the gradient of the input picture wrt this loss
grads = normalize(K.gradients(final_loss, input_tensor)[0])#####  エラー箇所
##本来は K.gradients(final_loss, input_tensor)[0] = Tensor("gradients_8/AddN_18:0", shape=(?, 224, 224, 3), dtype=float32) みたいな値を返すっぽい

# this function returns the loss and grads given the input picture
iterate = K.function([input_tensor], [loss1, loss2, loss3, loss1_neuron, loss2_neuron, loss3_neuron, grads])

# we run gradient ascent for 20 steps
for iters in range(args.grad_iterations):
    loss_value1, loss_value2, loss_value3, loss_neuron1, loss_neuron2, loss_neuron3, grads_value = iterate(
        [gen_img])
    if args.transformation == 'light':
        grads_value = constraint_light(grads_value)  # constraint the gradients value
    elif args.transformation == 'occl':
        grads_value = constraint_occl(grads_value, args.start_point,
                                      args.occlusion_size)  # constraint the gradients value
    elif args.transformation == 'blackout':
        grads_value = constraint_black(grads_value)  # constraint the gradients value

    gen_img += grads_value * args.step
    pred1, pred2, pred3 = model1.predict(gen_img), model2.predict(gen_img), model3.predict(gen_img)
    label1, label2, label3 = np.argmax(pred1[0]), np.argmax(pred2[0]), np.argmax(pred3[0])

    if not label1 == label2 == label3:
        update_coverage(gen_img, model1, model_layer_dict1, args.threshold)
        update_coverage(gen_img, model2, model_layer_dict2, args.threshold)
        update_coverage(gen_img, model3, model_layer_dict3, args.threshold)

        print(bcolors.OKGREEN + 'covered neurons percentage %d neurons %.3f, %d neurons %.3f, %d neurons %.3f'
              % (len(model_layer_dict1), neuron_covered(model_layer_dict1)[2], len(model_layer_dict2),
                  neuron_covered(model_layer_dict2)[2], len(model_layer_dict3),
                  neuron_covered(model_layer_dict3)[2]) + bcolors.ENDC)
        averaged_nc = (neuron_covered(model_layer_dict1)[0] + neuron_covered(model_layer_dict2)[0] +
                        neuron_covered(model_layer_dict3)[0]) / float(
            neuron_covered(model_layer_dict1)[1] + neuron_covered(model_layer_dict2)[1] +
            neuron_covered(model_layer_dict3)[
                1])
        print(bcolors.OKGREEN + 'averaged covered neurons %.3f' % averaged_nc + bcolors.ENDC)
        print('averaged covered neurons %.3f' % averaged_nc)

        gen_img_deprocessed = deprocess_image(gen_img)
        orig_img_deprocessed = deprocess_image(orig_img)

        # save the result to disk
        #imsave('./generated_inputs/' + args.transformation + '_' + decode_label(pred1) + '_' + decode_label(
        imageio.imwrite('./gdrive/My Drive/deepxplore/ImageNet/generated_inputs/' + args.transformation + '_' + decode_label(pred1) + '_' + decode_label(
            pred2) + '_' + decode_label(pred3) + '.png', gen_img_deprocessed)
        #imsave('./generated_inputs/' + args.transformation + '_' + decode_label(pred1) + '_' + decode_label(
        imageio.imwrite('./gdrive/My Drive/deepxplore/ImageNet/generated_inputs/' + args.transformation + '_' + decode_label(pred1) + '_' + decode_label(
          pred2) + '_' + decode_label(pred3) + '_orig.png', orig_img_deprocessed)
        break

In [0]:
%%time
for _ in range(args.seeds):
    gen_img = preprocess_image(random.choice(img_paths))
    orig_img = gen_img.copy()
    # first check if input already induces differences
    pred1, pred2, pred3 = model1.predict(gen_img), model2.predict(gen_img), model3.predict(gen_img)
    label1, label2, label3 = np.argmax(pred1[0]), np.argmax(pred2[0]), np.argmax(pred3[0])
    
    if not label1 == label2 == label3:
      ######################## 摂動を加えなくても予測が3つのモデルで異なる場合の処理 Start ########################
      print('input already causes different outputs: {}, {}, {}'.format(label1, label2 ,label3) + bcolors.ENDC) 

      update_coverage(gen_img, model1, model_layer_dict1, args.threshold)
      update_coverage(gen_img, model2, model_layer_dict2, args.threshold)
      update_coverage(gen_img, model3, model_layer_dict3, args.threshold)

      print(bcolors.OKGREEN + 'covered neurons percentage %d neurons %.3f, %d neurons %.3f, %d neurons %.3f'
      #        print('covered neurons percentage %d neurons %.3f, %d neurons %.3f, %d neurons %.3f'
            % (len(model_layer_dict1), neuron_covered(model_layer_dict1)[2], len(model_layer_dict2),
                neuron_covered(model_layer_dict2)[2], len(model_layer_dict3),
                neuron_covered(model_layer_dict3)[2]) + bcolors.ENDC)
      #                 neuron_covered(model_layer_dict3)[2]))

      averaged_nc = (neuron_covered(model_layer_dict1)[0] + neuron_covered(model_layer_dict2)[0] +
                      neuron_covered(model_layer_dict3)[0]) / float(
          neuron_covered(model_layer_dict1)[1] + neuron_covered(model_layer_dict2)[1] +
          neuron_covered(model_layer_dict3)[
              1])
      print(bcolors.OKGREEN + 'averaged covered neurons %.3f' % averaged_nc + bcolors.ENDC)
      print('averaged covered neurons %.3f' % averaged_nc)

      gen_img_deprocessed = deprocess_image(gen_img)

      # save the result to disk
      #imsave('./generated_inputs/' + 'already_differ_' + decode_label(pred1) + '_' + decode_label(
      imageio.imwrite('./gdrive/My Drive/deepxplore/ImageNet/generated_inputs/' + 'already_differ_' + str(label1) + '_' + str(label2) + '_' + str(label3) + '.png',
              gen_img_deprocessed)
      continue
      ######################## 摂動を加えなくても予測が3つのモデルで異なる場合の処理END ########################

    ######################## 予測が3つのモデルで同じ場合の処理 Start ########################
    # if all label agrees
    orig_label = label1
    layer_name1, index1 = neuron_to_cover(model_layer_dict1)
    layer_name2, index2 = neuron_to_cover(model_layer_dict2)
    layer_name3, index3 = neuron_to_cover(model_layer_dict3)
    # construct joint loss function
    if args.target_model == 0:
        loss1 = -args.weight_diff * K.mean(model1.get_layer('dense_7').output[..., orig_label])
        loss2 = K.mean(model2.get_layer('dense_8').output[..., orig_label])
        loss3 = K.mean(model3.get_layer('dense_9').output[..., orig_label])
    elif args.target_model == 1:
        loss1 = K.mean(model1.get_layer('dense_7').output[..., orig_label])
        loss2 = -args.weight_diff * K.mean(model2.get_layer('dense_8').output[..., orig_label])
        loss3 = K.mean(model3.get_layer('dense_9').output[..., orig_label])
    elif args.target_model == 2:
        loss1 = K.mean(model1.get_layer('dense_7').output[..., label1])
        loss2 = K.mean(model2.get_layer('dense_8').output[..., orig_label])
        loss3 = -args.weight_diff * K.mean(model3.get_layer('dense_9').output[..., orig_label])
    loss1_neuron = K.mean(model1.get_layer(layer_name1).output[..., index1])
    loss2_neuron = K.mean(model2.get_layer(layer_name2).output[..., index2])
    loss3_neuron = K.mean(model3.get_layer(layer_name3).output[..., index3])
    layer_output = (loss1 + loss2 + loss3) + args.weight_nc * (loss1_neuron + loss2_neuron + loss3_neuron)    

    # for adversarial image generation
    final_loss = K.mean(layer_output)

    # we compute the gradient of the input picture wrt this loss
    grads = normalize(K.gradients(final_loss, input_tensor)[0])#####  エラー箇所
    ##本来は K.gradients(final_loss, input_tensor)[0] = Tensor("gradients_8/AddN_18:0", shape=(?, 224, 224, 3), dtype=float32) みたいな値を返すっぽい

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_tensor], [loss1, loss2, loss3, loss1_neuron, loss2_neuron, loss3_neuron, grads])

    # we run gradient ascent for 20 steps
    for iters in range(args.grad_iterations):
        loss_value1, loss_value2, loss_value3, loss_neuron1, loss_neuron2, loss_neuron3, grads_value = iterate(
            [gen_img])
        if args.transformation == 'light':
            grads_value = constraint_light(grads_value)  # constraint the gradients value
        elif args.transformation == 'occl':
            grads_value = constraint_occl(grads_value, args.start_point,
                                          args.occlusion_size)  # constraint the gradients value
        elif args.transformation == 'blackout':
            grads_value = constraint_black(grads_value)  # constraint the gradients value

        gen_img += grads_value * args.step
        pred1, pred2, pred3 = model1.predict(gen_img), model2.predict(gen_img), model3.predict(gen_img)
        label1, label2, label3 = np.argmax(pred1[0]), np.argmax(pred2[0]), np.argmax(pred3[0])

        if not label1 == label2 == label3:
            update_coverage(gen_img, model1, model_layer_dict1, args.threshold)
            update_coverage(gen_img, model2, model_layer_dict2, args.threshold)
            update_coverage(gen_img, model3, model_layer_dict3, args.threshold)

            print(bcolors.OKGREEN + 'covered neurons percentage %d neurons %.3f, %d neurons %.3f, %d neurons %.3f'
                  % (len(model_layer_dict1), neuron_covered(model_layer_dict1)[2], len(model_layer_dict2),
                      neuron_covered(model_layer_dict2)[2], len(model_layer_dict3),
                      neuron_covered(model_layer_dict3)[2]) + bcolors.ENDC)
            averaged_nc = (neuron_covered(model_layer_dict1)[0] + neuron_covered(model_layer_dict2)[0] +
                            neuron_covered(model_layer_dict3)[0]) / float(
                neuron_covered(model_layer_dict1)[1] + neuron_covered(model_layer_dict2)[1] +
                neuron_covered(model_layer_dict3)[
                    1])
            print(bcolors.OKGREEN + 'averaged covered neurons %.3f' % averaged_nc + bcolors.ENDC)
            print('averaged covered neurons %.3f' % averaged_nc)

            gen_img_deprocessed = deprocess_image(gen_img)
            orig_img_deprocessed = deprocess_image(orig_img)

            # save the result to disk
            #imsave('./generated_inputs/' + args.transformation + '_' + decode_label(pred1) + '_' + decode_label(
            imageio.imwrite('./gdrive/My Drive/deepxplore/ImageNet/generated_inputs/' + args.transformation + '_' + str(label1) + '_' + str(label2) + '_' + str(label3) + '.png', gen_img_deprocessed)
            #imsave('./generated_inputs/' + args.transformation + '_' + decode_label(pred1) + '_' + decode_label(
            imageio.imwrite('./gdrive/My Drive/deepxplore/ImageNet/generated_inputs/' + args.transformation + '_' + str(label1) + '_' + str(label2) + '_' + str(label3) + '_orig.png', orig_img_deprocessed)
            break

input already causes different outputs: 0, 1, 1
covered neurons percentage 13890 neurons 0.624, 13890 neurons 0.623, 13890 neurons 0.623
averaged covered neurons 0.624
averaged covered neurons 0.624
input already causes different outputs: 0, 1, 1
covered neurons percentage 13890 neurons 0.625, 13890 neurons 0.625, 13890 neurons 0.625
averaged covered neurons 0.625
averaged covered neurons 0.625
input already causes different outputs: 0, 1, 1
covered neurons percentage 13890 neurons 0.632, 13890 neurons 0.632, 13890 neurons 0.632
averaged covered neurons 0.632
averaged covered neurons 0.632
input already causes different outputs: 1, 0, 0
covered neurons percentage 13890 neurons 0.634, 13890 neurons 0.634, 13890 neurons 0.634
averaged covered neurons 0.634
averaged covered neurons 0.634
input already causes different outputs: 0, 1, 1
covered neurons percentage 13890 neurons 0.635, 13890 neurons 0.635, 13890 neurons 0.635
averaged covered neurons 0.635
averaged covered neurons 0.635
input

## エラー箇所
* grads = normalize(K.gradients(final_loss, input_tensor)[0])
## 原因 
* K.gradients(final_loss, input_tensor)[0] が None

## 追跡

##小澤さんのコードと異なる部分
### layer_output が異なる
#### Nomal
```
layer_output
Tensor("add_20:0", shape=(), dtype=float32)
```
### Error
```
layer_output
Tensor("add_50:0", shape=(), dtype=float32)
```


### final_loss も異なる
（ final_loss = K.mean(layer_output) なので当然ではあるが。）
#### Nomal
```
final_loss
Tensor("Mean_6:0", shape=(), dtype=float32)

```
#### Error
```
final_loss
Tensor("Mean_48:0", shape=(), dtype=float32)
```


### input_tensor は同じ
#### Nomal
```
input_tensor
Tensor("input_1:0", shape=(?, 224, 224, 3), dtype=float32)
```
#### Error
```
input_tensor
Tensor("input_1:0", shape=(?, 224, 224, 3), dtype=float32)
```

### layer_output は以下で定義
```
layer_output = (loss1 + loss2 + loss3) + args.weight_nc * (loss1_neuron + loss2_neuron + loss3_neuron)
```
loss1, loss2, loss3, loss1_neuron, loss2_neuron, loss3_neuron は以下で定義
```
    if args.target_model == 0:
        loss1 = -args.weight_diff * K.mean(model1.get_layer('predictions').output[..., orig_label])
        loss2 = K.mean(model2.get_layer('predictions').output[..., orig_label])
        loss3 = K.mean(model3.get_layer('fc1000').output[..., orig_label])
    elif args.target_model == 1:
        loss1 = K.mean(model1.get_layer('predictions').output[..., orig_label])
        loss2 = -args.weight_diff * K.mean(model2.get_layer('predictions').output[..., orig_label])
        loss3 = K.mean(model3.get_layer('fc1000').output[..., orig_label])
    elif args.target_model == 2:
        loss1 = K.mean(model1.get_layer('predictions').output[..., label1])
        loss2 = K.mean(model2.get_layer('predictions').output[..., orig_label])
        loss3 = -args.weight_diff * K.mean(model3.get_layer('fc1000').output[..., orig_label])
    loss1_neuron = K.mean(model1.get_layer(layer_name1).output[..., index1])
    loss2_neuron = K.mean(model2.get_layer(layer_name2).output[..., index2])
    loss3_neuron = K.mean(model3.get_layer(layer_name3).output[..., index3])
    layer_output = (loss1 + loss2 + loss3) + args.weight_nc * (loss1_neuron + loss2_neuron + loss3_neuron)
```

### loss1～3, loss1～3_neuron の差分

### Nomal
```
loss1
Tensor("mul_118:0", shape=(), dtype=float32)
loss2
Tensor("Mean_473:0", shape=(), dtype=float32)
loss3
Tensor("Mean_474:0", shape=(), dtype=float32)
loss1_neuron
Tensor("Mean_475:0", shape=(), dtype=float32)
loss2_neuron
Tensor("Mean_476:0", shape=(), dtype=float32)
loss3_neuron
Tensor("Mean_477:0", shape=(), dtype=float32)
layer_output
Tensor("add_374:0", shape=(), dtype=float32)
```

### Error
```
loss1
Tensor("mul_14:0", shape=(), dtype=float32)
loss2
Tensor("Mean_50:0", shape=(), dtype=float32)
loss3
Tensor("Mean_51:0", shape=(), dtype=float32)
loss1_neuron
Tensor("Mean_52:0", shape=(), dtype=float32)
loss2_neuron
Tensor("Mean_53:0", shape=(), dtype=float32)
loss3_neuron
Tensor("Mean_54:0", shape=(), dtype=float32)
layer_output
Tensor("add_55:0", shape=(), dtype=float32)
```


